<a href="https://colab.research.google.com/github/RebeccaBZ/Hotel_Booking_Case_Study/blob/main/WebScraping_booking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import re

url = 'https://www.booking.com/searchresults.en-gb.html?ss=Beirut&ssne=Beirut&ssne_untouched=Beirut&highlighted_hotels=5416928&efdco=1&label=gen173nr-1FCAMoggFCBmJlaXJ1dEgJWARoggGIAQGYAQm4ARfIAQzYAQHoAQH4AQKIAgGoAgO4AsibjKgGwAIB0gIkMTcyNDkzZmUtZWExNS00MThjLWI2OGUtODRmMmI2ZmZiNTI22AIF4AIB&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=-801546&dest_type=city&ltfd=1%3A5%3A8-2024%3A1%3A&group_adults=2&no_rooms=1'
headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246',
      'Accept-Language': 'en-US, en;q=0.5'
  }

# convert to usd
def convert_price(price_text):
    # choose only the digits
    filtered_price = re.sub(r'[^\d.]', '', price_text)

    conversion_rate = 0.0585
    converted_price = float(filtered_price) * conversion_rate

    return converted_price

'''
# convert to usd
def convert_price_2(price_text):
    # choose only the digits
    filtered_price = re.sub(r'\d+', '', price_text)

    conversion_rate = 0.0585
    converted_price = float(filtered_price) * conversion_rate

    return converted_price
'''

def extractData():
  # Find the filtered hotels in Beirut
  hotels_beirut = soup.findAll('div', {'data-testid': 'property-card'})

  for hotel_beirut in hotels_beirut:
    #get every hotel's name
    hotel_name = hotel_beirut.find('div', {"data-testid": "title"})
    name_per_hotel = hotel_name.text.strip()

    #get every hotel's location
    hotel_location = hotel_beirut.find('span', {"data-testid": "address"})
    location_per_hotel = hotel_location.text.strip()

    #get every hotel's distance from the center of Beirut
    hotel_distance_beirut = hotel_beirut.find('span', {"data-testid": "distance"})
    distance_beirut_per_hotel = hotel_distance_beirut.text.strip()

    #get every hotel's number of reviews
    hotel_nb_reviews = hotel_beirut.find('div', {"class": "abf093bdfe f45d8e4c32 d935416c47"})
    nb_reviews_per_hotel = hotel_nb_reviews.text.strip()

    #get every hotel's distance from the beach
    hotel_beach = hotel_beirut.find('span', {"class":"abf093bdfe a196e30dac"})

    if hotel_beach:
      beach_per_hotel = hotel_beach.text.strip()
    else:
      beach_per_hotel = "N/A"

    # beach_per_hotel = hotel_beach.text.strip()

    #get every hotel's sustainable implementations
    hotel_sustainability= hotel_beirut.find('span', {"class":"abf093bdfe d068504c75 be09c104ad"})

    if hotel_sustainability:
      sus_per_hotel = hotel_sustainability.text.strip()
    else:
      sus_per_hotel = "N/A"

    #get every hotel's room type
    hotel_room_type= hotel_beirut.find('span', {"class":"df597226dd"})

    if hotel_room_type:
      room_type_per_hotel = hotel_room_type.text.strip()
    else:
      room_type_per_hotel = "N/A"

    #get every hotel's room cancellation policy
    hotel_cancellation_policy=hotel_beirut.find('div', {"class":"abf093bdfe d068504c75"})

    if hotel_cancellation_policy:
      cancellation_per_hotel = hotel_cancellation_policy.text.strip()
    else:
      cancellation_per_hotel = "N/A"


    #get every hotel's room policies
    hotel_policy_elements = hotel_beirut.find_all('div', {"class": "abf093bdfe d068504c75"})

    #create a list to store policies
    policies_per_hotel = []

    for policy_element in hotel_policy_elements:
        policy_text = policy_element.text.strip()
        policies_per_hotel.append(policy_text)

    #get every hotel's rating
    hotel_rating= hotel_beirut.find('div', {"class":"a3b8729ab1 d86cee9b25"})

    if hotel_rating:
      rating_per_hotel = hotel_rating.text.strip()
    else:
      rating_per_hotel = "N/A"

    #get every hotel's rating title
    hotel_rating_title = hotel_beirut.find('div', {"class": "a3b8729ab1 e6208ee469 cb2cbb3ccb"})
    rating_title_per_hotel = hotel_rating_title.text.strip()

    #get every hotel's price for the filtered values
    hotel_price = hotel_beirut.find('span', {"data-testid":"price-and-discounted-price"})

    if hotel_price:
      price_per_hotel_edit = hotel_price.text.strip()
      price_per_hotel = convert_price(price_per_hotel_edit)
    else:
      price_per_hotel = "N/A"

    #get every hotel's additional charges including taxes
    hotel_add_charges = hotel_beirut.find('div', {"data-testid":"taxes-and-charges"})

    if hotel_add_charges:
      charges_per_hotel = hotel_add_charges.text.strip()
    else:
      charges_per_hotel = "N/A"

    # get link of every hotel in case further details are needed by the users
    hotel_link = hotel_beirut.find('a', {"data-testid":"title-link"})

    if hotel_link:
      link_per_hotel = hotel_link.get('href')


    # Append the info about every hotel to the empty list created
    info_per_hotel.append({
    'hotel_name': name_per_hotel,
    'location': location_per_hotel,
    'distance_from_beirut': distance_beirut_per_hotel,
    'number_reviews':nb_reviews_per_hotel,
    'distance_from_beach': beach_per_hotel,
    'sustainability_level': sus_per_hotel,
    'room_type':room_type_per_hotel,
    'cancellation_policy': cancellation_per_hotel,
    'hotel_policies': policies_per_hotel,
    'rating': rating_per_hotel,
    'rating_tile': rating_title_per_hotel,
    'price': price_per_hotel,
    'additional_charges': charges_per_hotel,
    'link': link_per_hotel

      })

response  = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# get the info for every hotel
info_per_hotel = []
extractData()

button = soup.find('button', {"class":"a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 deab83296e bb803d8689 a16ddf9c57"})


# Check if the button is disabled
if button and 'disabled' in button.attrs:
    sys.exit()
else:
    url= 'https://www.booking.com/searchresults.en-gb.html?label=gen173nr-1FCAMoggFCBmJlaXJ1dEgJWARoggGIAQGYAQm4ARfIAQzYAQHoAQH4AQKIAgGoAgO4AsibjKgGwAIB0gIkMTcyNDkzZmUtZWExNS00MThjLWI2OGUtODRmMmI2ZmZiNTI22AIF4AIB&aid=304142&ss=Beirut&ssne=Beirut&ssne_untouched=Beirut&highlighted_hotels=5416928&efdco=1&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=-801546&dest_type=city&ltfd=1%3A5%3A8-2024%3A1%3A&group_adults=2&no_rooms=1&offset=25'
    headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246',
      'Accept-Language': 'en-US, en;q=0.5'
    }
    response  = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    extractData()

hotel_booking = pd.DataFrame(info_per_hotel)
# hotel_booking.head()

hotel_booking.to_csv('hotels_booking.csv', header=True, index=False)


In [ ]:
info_per_hotel

[{'hotel_name': 'Rawsheh 51',
  'location': 'Raouche, Beirut',
  'distance_from_beirut': '2.4 km from centre',
  'number_reviews': '1,533 reviews',
  'distance_from_beach': '450 m from beach',
  'sustainability_level': 'N/A',
  'room_type': 'N/A',
  'cancellation_policy': 'N/A',
  'hotel_policies': [],
  'rating': '7.8',
  'rating_tile': 'Good',
  'price': 'N/A',
  'additional_charges': 'N/A',
  'link': 'https://www.booking.com/hotel/lb/rawsheh-51.en-gb.html?label=gen173nr-1FCAMoggFCBmJlaXJ1dEgJWARoggGIAQGYAQm4ARfIAQzYAQHoAQH4AQKIAgGoAgO4AsibjKgGwAIB0gIkMTcyNDkzZmUtZWExNS00MThjLWI2OGUtODRmMmI2ZmZiNTI22AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-08-05&checkout=2024-08-10&dest_id=-801546&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=d2586828f19b01c7&srepoch=1694962130&soh=1&from_sustainable_property_sr=1&from=searchresults#no_availability_msg'},
 {'hotel_name': 'Four Points By Sheraton Le Verdun',
  'lo

[<div class="c82435a4b8 a178069f51 a6ae3c2b40 a18aeea94d d794b7a0f7 f53e278e95 da89aeb942 c12ee2f811" data-soldout="1" data-testid="property-card" style="--bui_box_padding--s:4"><div class="d20f4628d0"><div class="f9d4f2568d"><div class="c90a25d457"><a aria-hidden="true" data-testid="property-card-desktop-single-image" href="https://www.booking.com/hotel/lb/rawsheh-51.en-gb.html?label=gen173nr-1FCAMoggFCBmJlaXJ1dEgJWARoggGIAQGYAQm4ARfIAQzYAQHoAQH4AQKIAgGoAgO4AsibjKgGwAIB0gIkMTcyNDkzZmUtZWExNS00MThjLWI2OGUtODRmMmI2ZmZiNTI22AIF4AIB&amp;aid=304142&amp;ucfs=1&amp;arphpl=1&amp;checkin=2024-08-05&amp;checkout=2024-08-10&amp;dest_id=-801546&amp;dest_type=city&amp;group_adults=2&amp;req_adults=2&amp;no_rooms=1&amp;group_children=0&amp;req_children=0&amp;hpos=1&amp;hapos=1&amp;sr_order=popularity&amp;srpvid=c960485acaae0015&amp;srepoch=1694859447&amp;soh=1&amp;from_sustainable_property_sr=1&amp;from=searchresults#no_availability_msg" rel="noopener noreferrer" tabindex="-1" target="_blank"><img 